In [1]:
from pyspark.sql import SparkSession

# create a SparkSession
spark = SparkSession.builder.appName("ReadJSON").getOrCreate()
# .config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:3.0.1")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/01 20:37:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
from pyspark.sql.types import StructType, StructField, StringType, FloatType, BooleanType, IntegerType
#"reviewerID": "A8WEXFRWX1ZHH", 
# "asin": "0209688726", 
# "style": {"Color:": " AC"}, 
# "reviewerName": "Goldengate",
# Define the schema
schema = StructType([
    StructField("overall", FloatType(), True),
    StructField("verified", BooleanType(), True),
    StructField("reviewTime", StringType(), True),
    StructField("reviewerID", StringType(), True),
    StructField("asin", StringType(), True),
    StructField("style", StructType([StructField("Color:", StringType(), True)]), True),
    StructField("reviewerName", StringType(), True),
    StructField("reviewText", StringType(), True),
    StructField("unixReviewTime", IntegerType(), True)
    
])

In [3]:
df = spark.read.schema(schema).json("/Users/kravisankaran/Downloads/Automotive_5.json")

In [4]:
df.show()

+-------+--------+-----------+--------------+----------+--------+--------------------+--------------------+--------------+
|overall|verified| reviewTime|    reviewerID|      asin|   style|        reviewerName|          reviewText|unixReviewTime|
+-------+--------+-----------+--------------+----------+--------+--------------------+--------------------+--------------+
|    4.0|   false| 05 1, 2015| A8WEXFRWX1ZHH|0209688726|   { AC}|          Goldengate|After I wrote the...|    1430438400|
|    1.0|    true|04 19, 2018| ABCA1A8E4DGV1|0209688726| { Blue}|                 noe|It sucks barely p...|    1524096000|
|    1.0|    true|04 16, 2018|A1NX8HM89FRQ32|0209688726|{ Black}|              Eduard|Well to write a s...|    1523836800|
|    3.0|    true|04 13, 2018|A1X77G023NY0KY|0209688726|   { CA}|              Lauren|I have absolutely...|    1523577600|
|    5.0|    true| 04 8, 2018|A3GK37JO2MGW6Q|0209688726|{ Black}|               danny|it ok it does it job|    1523145600|
|    5.0|    tru

In [5]:
from pyspark.sql.functions import col, struct
df_modified = df.withColumn("style", struct(col("style.Color:").alias("Color")))

In [6]:
from pyspark.sql.functions import to_date
df_with_date = df.withColumn("reviewTime", to_date(df.reviewTime, "MM d, yyyy"))

In [7]:
df_with_date.show()

+-------+--------+----------+--------------+----------+--------+--------------------+--------------------+--------------+
|overall|verified|reviewTime|    reviewerID|      asin|   style|        reviewerName|          reviewText|unixReviewTime|
+-------+--------+----------+--------------+----------+--------+--------------------+--------------------+--------------+
|    4.0|   false|2015-05-01| A8WEXFRWX1ZHH|0209688726|   { AC}|          Goldengate|After I wrote the...|    1430438400|
|    1.0|    true|2018-04-19| ABCA1A8E4DGV1|0209688726| { Blue}|                 noe|It sucks barely p...|    1524096000|
|    1.0|    true|2018-04-16|A1NX8HM89FRQ32|0209688726|{ Black}|              Eduard|Well to write a s...|    1523836800|
|    3.0|    true|2018-04-13|A1X77G023NY0KY|0209688726|   { CA}|              Lauren|I have absolutely...|    1523577600|
|    5.0|    true|2018-04-08|A3GK37JO2MGW6Q|0209688726|{ Black}|               danny|it ok it does it job|    1523145600|
|    5.0|    true|2018-0

In [8]:
import pyspark.sql.functions as F
from pyspark.sql.types import StructType, StructField, StringType, FloatType, BooleanType, IntegerType, ArrayType
# targetUDF = F.udf(lambda x: 1 if x >= 4.0 else (0 if x == 3.0 else -1), IntegerType())
targetUDF = F.udf(lambda x: 1 if x >= 4.0 else 0, IntegerType())
import re
import nltk
from nltk.corpus import stopwords

In [9]:
def preProcess(text):
    # Should return a list of tokens
    text = re.sub(r"(\w)([.,;:!?'\"”\)])", r"\1 \2", text)
    text = re.sub(r"([.,;:!?'\"“\(])(\w)", r"\1 \2", text)    
    text = text.lower()
    tokens = word_tokenize(text)    
    return tokens

In [10]:
df_sentiment = df_with_date.withColumn("sentiment", targetUDF(df_with_date["overall"]))

In [11]:
df_sentiment.show()

24/04/01 20:37:45 ERROR Executor: Exception in task 0.0 in stage 2.0 (TID 2)/ 1]
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/homebrew/Cellar/apache-spark/3.5.1/libexec/python/lib/pyspark.zip/pyspark/worker.py", line 1100, in main
    raise PySparkRuntimeError(
pyspark.errors.exceptions.base.PySparkRuntimeError: [PYTHON_VERSION_MISMATCH] Python in worker has different version (3, 9) than that in driver 3.12, PySpark cannot run with different minor versions.
Please check environment variables PYSPARK_PYTHON and PYSPARK_DRIVER_PYTHON are correctly set.

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.scala:572)
	at org.apache.spark.sql.execution.python.BasePythonUDFRunner$$anon$1.read(PythonUDFRunner.scala:94)
	at org.apache.spark.sql.execution.python.BasePythonUDFRunner$$anon$1.read(PythonUDFRunner.scala:75)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunn

PythonException: 
  An exception was thrown from the Python worker. Please see the stack trace below.
Traceback (most recent call last):
  File "/opt/homebrew/Cellar/apache-spark/3.5.1/libexec/python/lib/pyspark.zip/pyspark/worker.py", line 1100, in main
    raise PySparkRuntimeError(
pyspark.errors.exceptions.base.PySparkRuntimeError: [PYTHON_VERSION_MISMATCH] Python in worker has different version (3, 9) than that in driver 3.12, PySpark cannot run with different minor versions.
Please check environment variables PYSPARK_PYTHON and PYSPARK_DRIVER_PYTHON are correctly set.


In [ ]:
from pyspark.ml.feature import Tokenizer

# use PySparks build in tokenizer to tokenize tweets
tokenizer = Tokenizer(inputCol  = "reviewText",
                      outputCol = "token")
df4 = tokenizer.transform(df_sentiment.filter(df.reviewText.isNotNull()))

In [ ]:
df4.show()

In [ ]:
import re

def removeRegex(tokens: list) -> list:
    """
    Removes hashtags, call outs and web addresses from tokens.
    """
    expr    = '(@[A-Za-z0-a9_]+)|(#[A-Za-z0-9_]+)|'+\
              '(https?://[^\s<>"]+|www\.[^\s<>"]+)'
        
    regex   = re.compile(expr)

    cleaned = [t for t in tokens if not(regex.search(t)) if len(t) > 0]

    return list(filter(None, cleaned))

In [ ]:
removeWEBUDF = F.udf(removeRegex, ArrayType(StringType()))

In [ ]:
def normalize(tokens : list) -> list:
    """
    Removes non-english characters and returns lower case versions of words.
    """
    subbed   = [re.sub("[^a-zA-Z]+", "", s).lower() for s in tokens]
    
    filtered = filter(None, subbed)
    
    return list(filtered)


normalizeUDF = F.udf(normalize, ArrayType(StringType()))

In [ ]:
# remove hashtags, call outs and web addresses
df4 = df4.withColumn("tokens_re", removeWEBUDF(df4["token"]))

# remove non english characters
df4 = df4.withColumn("tokens_clean", normalizeUDF(df4["tokens_re"]))

# rename columns
df5 = df4.drop("token","tokens_re")
df5 = df5.withColumnRenamed("tokens_clean", "tokens")\

# remove reviews where the tokens array is empty, i.e. where it was just
# a hashtag, callout, numbers, web adress etc.
df6 = df5.where(F.size(F.col("tokens")) > 0)

In [ ]:
df6.limit(2).toPandas()

In [ ]:
# db_name = "poc"
# collection_name = "reviews"
# df_sentiment.write.format("mongo").option("uri", "mongodb://localhost:27017/poc.reviews").mode("overwrite").save()





# import pymongo



# conn = pymongo.MongoClient('mongodb://localhost:27017')
# db = conn.poc

In [ ]:
# reviews = db.reviews
# query = {"asin": "0209688726"}
# # for res in results:
# #     print("Document = {}\n".format(res))

In [ ]:
# count_sentiment = {"$group": 
#                      {"_id" : {"sentiment":"$sentiment"},  # note use a $ on the field
#                       "ct"  : {"$sum":1}
#                      }
#                   }
# results = reviews.aggregate([count_sentiment], allowDiskUse=True)

# for res in results:
#     print(res)

In [ ]:
df7 = df6.select("reviewText","sentiment")\
        .withColumnRenamed("sentiment", "label")

In [ ]:
train, test = df7.randomSplit([0.80, 0.20], 1234)

In [ ]:
train.cache()

In [ ]:
train.groupby("label")\
     .count()\
     .show()

In [ ]:
test.cache()

In [ ]:
test.groupby("label")\
    .count()\
    .show()

In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import Tokenizer, HashingTF, IDF
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [ ]:
evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction")

# get the name of the metric used
evaluator.getMetricName()

In [ ]:
# create tokens from tweets
tk = Tokenizer(inputCol= "reviewText", outputCol = "tokens")

# create term frequencies for each of the tokens
tf1 = HashingTF(inputCol="tokens", outputCol="rawFeatures", numFeatures=1e5)

# create tf-idf for each of the tokens
idf = IDF(inputCol="rawFeatures", outputCol="features", minDocFreq=2.0)

# create basic logistic regression model
lr = LogisticRegression(maxIter=20)

# create entire pipeline
basic_pipeline = Pipeline(stages=[tk, tf1, idf, lr])

In [ ]:
model1         = basic_pipeline.fit(train)

In [ ]:
# predict on test set
predictions1   = model1.transform(test)

# get the performance on the test set
score1         = evaluator.evaluate(predictions1)

print("AUC SCORE: {}".format(score1))

In [ ]:
predictedAndLabels = predictions1.select(["prediction","label"])\
                                 .rdd.map(lambda r : (float(r[0]), float(r[1])))

from pyspark.mllib.evaluation import MulticlassMetrics

metrics = MulticlassMetrics(predictedAndLabels)

print("Test Set Accuracy: {}".format(metrics.accuracy))

In [ ]:
from pyspark.ml.feature import StopWordsRemover

In [ ]:
sw  = StopWordsRemover(inputCol="tokens", outputCol="filtered")
tf2 = HashingTF(inputCol="filtered", outputCol="rawFeatures", numFeatures=1e5)

In [ ]:
sw_pipleline  = Pipeline(stages=[tk, sw, tf2, idf, lr])

model2        = sw_pipleline.fit(train)
predictions2  = model2.transform(test)
score2        = evaluator.evaluate(predictions2)

print("AUC SCORE: {}".format(score2))

In [ ]:
import nltk
print(nltk.__version__)
from nltk.stem.porter import PorterStemmer

In [ ]:
# Testing
stemmer = PorterStemmer()

tokens  = "my feelings having studied all day".split(" ")
print("raw tokens: {}".format(tokens))

In [ ]:
tokens_stemmed = [stemmer.stem(token) for token in tokens]
print("clean tokens: {}".format(tokens_stemmed))

In [ ]:
from pyspark import keyword_only
import numpy as np
import pyspark.sql.functions as F
from pyspark.sql import DataFrame
from pyspark.sql.types import ArrayType, StringType
from pyspark.ml import Transformer
from pyspark.ml.param.shared import HasInputCol, HasOutputCol, Param


class PorterStemming(Transformer, HasInputCol, HasOutputCol):
    """
    PosterStemming class using the NLTK Porter Stemmer
    
    This comes from https://stackoverflow.com/questions/32331848/create-a-custom-transformer-in-pyspark-ml
    Adapted to work with the Porter Stemmer from NLTK.
    """
    
    @keyword_only
    def __init__(self, 
                 inputCol  : str = None, 
                 outputCol : str = None, 
                 min_size  : int = None):
        """
        Constructor takes in the input column name, output column name,
        plus the minimum legnth of a token (min_size)
        """
        # call Transformer classes constructor since were extending it.
        super(Transformer, self).__init__()

        # set Parameter objects minimum token size
        self.min_size = Param(self, "min_size", "")
        self._setDefault(min_size=0)

        # set the input keywork arguments
        kwargs = self._input_kwargs
        self.setParams(**kwargs)

        # initialize Stemmer object
        self.stemmer  = PorterStemmer()

        
    @keyword_only
    def setParams(self, 
                  inputCol  : str = None, 
                  outputCol : str = None, 
                  min_size  : int = None
      ) -> None:
        """
        Function to set the keyword arguemnts
        """
        kwargs = self._input_kwargs
        return self._set(**kwargs)
    

    def _stem_func(self, words  : list) -> list:
        """
        Stemmer function call that performs stemming on a
        list of tokens in words and returns a list of tokens
        that have meet the minimum length requiremnt.
        """
        # We need a way to get min_size and cannot access it 
        # with self.min_size
        min_size       = self.getMinSize()

        # stem that actual tokens by applying 
        # self.stemmer.stem function to each token in 
        # the words list
        stemmed_words  = map(self.stemmer.stem, words)

        # now create the new list of tokens from
        # stemmed_words by filtering out those
        # that are not of legnth > min_size
        filtered_words = filter(lambda x: len(x) > min_size, stemmed_words)

        return list(filtered_words)
    
    def _transform(self, df: DataFrame) -> DataFrame:
        """
        Transform function is the method that is called in the 
        MLPipleline.  We have to override this function for our own use
        and have it call the _stem_func.

        Notice how it takes in a type DataFrame and returns type Dataframe
        """
        # Get the names of the input and output columns to use
        out_col       = self.getOutputCol()
        in_col        = self.getInputCol()

        # create the stemming function UDF by wrapping the stemmer 
        # method function
        stem_func_udf = F.udf(self._stem_func, ArrayType(StringType()))
        
        # now apply that UDF to the column in the dataframe to return
        # a new column that has the same list of words after being stemmed
        df2           = df.withColumn(out_col, stem_func_udf(df[in_col]))

        return df2
  
  
    def setMinSize(self,value):
        """
        This method sets the minimum size value
        for the _paramMap dictionary.
        """
        self._paramMap[self.min_size] = value
        return self

    def getMinSize(self) -> int:
        """
        This method uses the parent classes (Transformer)
        .getOrDefault method to get the minimum
        size of a token.
        """
        return self.getOrDefault(self.min_size)


In [ ]:
stem2 = PorterStemming(inputCol="tokens", outputCol="stemmed")


In [ ]:
stem_pipeline = Pipeline(stages= [tk, stem2]).fit(train)

In [ ]:
train_stem = stem_pipeline.transform(train)\
                          .where(F.size(F.col("stemmed")) >= 1)
train_stem.show()

# test_stem  = stem_pipeline.transform(test)\
#                           .where(F.size(F.col("stemmed")) >= 1)

# # cache them to avoid running stemming 
# # each iteration in the grid search
# train_stem.cache()
# test_stem.cache()

In [ ]:
# test_stem.show(5)

In [ ]:
from pyspark.ml.feature import NGram

bigram = NGram(inputCol="tokens", outputCol="bigrams", n=2)

In [ ]:
tf5   = HashingTF(inputCol="bigrams", outputCol="rawFeatures", numFeatures=2e5)

bigram_pipeline  = Pipeline(stages= [tk, bigram, tf5, idf, lr])

model5           = bigram_pipeline.fit(train)
predictions5     = model5.transform(test)

score5           = evaluator.evaluate(predictions5)

In [ ]:
print("AUC SCORE: {}".format(score5))

In [ ]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize

stemmer = PorterStemmer()

def stem_text(text):
    tokens = word_tokenize(text)
    stemmed_tokens = [stemmer.stem(token) for token in tokens]
    return " ".join(stemmed_tokens)

# Create a UDF
stem_text_udf = udf(stem_text, StringType())

In [ ]:
bigram2 = NGram(inputCol="stemmed", outputCol="bigrams", n=2)

tf6     = HashingTF(inputCol="bigrams", outputCol="rawFeatures", numFeatures=2e5)

idf     = IDF(inputCol="rawFeatures", outputCol="features", minDocFreq=2.0)

lr      = LogisticRegression(maxIter=20)

stem_bigram_pipeline  = Pipeline(stages= [bigram2, tf6, idf, lr])

model6                = stem_bigram_pipeline.fit(train_stem)
predictions6          = model6.transform(test_stem)

score6                = evaluator.evaluate(predictions6)